In [14]:
import requests
import time
import numpy as np
import json

# Utils for timing

In [12]:
time.time()

1701553203.558945

In [21]:
def average_duration(request: str, data_gen=str, method:str='GET', iterations: int=30000) -> float:
    '''
        Returns duration of the requests in s.
    '''

    if method == 'GET':
        rec_function = requests.get
    elif method == 'POST':
        rec_function = requests.post
    elif method == 'DELETE':
        rec_function = requests.delete
    elif method == 'UPDATE':
        rec_function = requests.patch
    else:
        raise ValueError('Method has not been recognized!')
    
    durations = []
    for _ in range(iterations):
        start = time.time()
        rec_function(url=request, data=next(data_gen), headers={'Content-type': 'application/json'})
        end = time.time()

        durations.append(end - start)

    print('On average request takes: ', np.mean(durations))

# Tests of NoSQL: MongoDB

# Tests of SQL: PostgreSQL

## Insert

In [22]:
request = 'http://localhost:5051/users/insert'

def generate_user(num: int):
    start = 9281497

    for i in range(num):
        data = json.dumps({
            'telegram_id' : start + i,
            'username': 'varenik',
            'about': '''On Unix, return the current processor time as a floating point number expressed in seconds. The precision, and in fact the very definition of the meaning of “processor time”, depends on that of the C function of the same name. On Windows, this function returns wall-clock seconds elapsed since the first call to this function, as a floating point number, based on the Win32 function QueryPerformanceCounter(). The resolution is typically better than one microsecond.'''
        })

        yield data

In [23]:
requests_num = 30000

average_duration(request=request, data_gen=iter(generate_user(requests_num)), method='POST', iterations=requests_num)

On average request takes:  0.003915661104520162


## Extract

In [26]:
request = 'http://localhost:5051/users'

def generate_empty(num: int):
    for _ in range(num):
        yield json.dumps({})

In [27]:
requests_num = 300

average_duration(request=request, data_gen=iter(generate_empty(requests_num)), method='GET', iterations=requests_num)

On average request takes:  0.15078813552856446


## Update

## Delete